# Amazon Bedrock 上の Claude Code: クイック セットアップ ガイド

このノートブックには、Claude 3.7 Sonnet モデルを搭載した AWS Bedrock を使用して、Anthropic の AI コーディング アシスタントである Claude Code を構成するためのガイドが記載されています。

## Claude Code とは?

Claude Code は、ターミナルベースの強力なコーディング アシスタントです。次の機能があります:
- コードベース全体でファイルを編集し、バグを修正します
- コードのアーキテクチャとロジックに関する質問に答えます
- テストを実行して修正し、マージの競合を解決します
- ローカル環境でターミナルから直接操作します

## Claude コード モデル

Claude コードでは、Claude 3.7 Sonnet モデルと Claude 3.5 Haiku モデルの両方にアクセスする必要があります:

- **Claude 3.7 Sonnet**: 複雑なコードの理解、生成、推論タスクを処理する主要なモデル

- **Claude 3.5 Haiku**: Claude コード内の特定の機能に使用される軽量モデル:
- より単純なクエリの応答時間が短縮されます
- 日常的な操作でのトークンの使用効率が向上します
- バックグラウンド タスクとファイルのインデックス作成を処理します
- より強力な Sonnet モデルに送信する前にコードを前処理してフィルタリングします

このデュアル モデル アプローチにより、パフォーマンスとコストの両方が最適化されます。適切な場所で軽量の Haiku モデルを使用すると、応答性を維持しながら全体的なトークン消費量を削減でき、より強力な Sonnet モデルは、高度な機能を必要とする複雑なコーディング タスクに確保できます。

> ⚠️ **重要** ⚠️: 使用しているリージョンの AWS Bedrock モデルアクセス設定で、Claude 3.7 Sonnet モデルと Claude 3.5 Haiku モデルの **両方** へのアクセスを明示的に有効にする必要があります。どちらかのモデルを有効にしないと、Claude Code の使用時に 403 Forbidden エラーが発生します。🔑 🚫

> モデルアクセスを有効にする手順については、[AWS Bedrock 入門ガイド](https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html) を参照してください。

## Claude 3.7 Sonnet について

Claude 3.7 Sonnet は、ソフトウェア開発タスク向けに最適化された Anthropic の最新モデルです。
- 優れたコード理解および生成機能
- 複雑な問題のデバッグのための推論機能の強化
- 大規模なコードベースをナビゲートするためのコンテキスト処理の改善
- プログラミングの概念とパターンの理解の向上
- Claude Code でのターミナルベースのインタラクション向けに最適化

この強力なモデルは、パフォーマンスとコストのバランスを維持しながら、コーディング支援を大幅に改善します。

## Amazon Bedrock で Claude Code を使用する利点

- 既存の AWS Bedrock 割り当てと課金を活用
- Claude 3.7 Sonnet の高度なコーディング機能にアクセス
- AWS インフラストラクチャ内でコード操作を維持
- 適切に構成すると、より高いレート制限にアクセスできる可能性あり
- 一貫したセキュリティとコンプライアンス標準を維持
- コスト最適化のためにプロンプ​​ト キャッシュ (使用可能な場合) を使用

## 始める前に

### システム要件を確認してください
- **オペレーティング システム**: macOS 10.15+、Ubuntu 20.04+/Debian 10+、または WSL 経由の Windows
- **ハードウェア**: 最低 4GB RAM
- **ソフトウェア**:
- Node.js 18+
- git 2.23+ (オプション)
- PR ワークフロー用の GitHub または GitLab CLI (オプション)
- 拡張ファイル検索用の ripgrep (rg) (オプション)
- **ネットワーク**: 認証と AI 処理にはインターネット接続が必要です

### AWS 認証情報の設定

開始する前に、AWS 認証情報が適切に設定されていることを確認してください。Claude Code は、これらの認証情報を使用して Amazon Bedrock にアクセスします。

AWS 認証情報の設定に関する詳細な手順については、https://docs.aws.amazon.com/cli/v1/userguide/cli-chap-configure.html を参照してください。

#### クイック AWS 認証情報の設定:
- **AWS CLI の使用** (最も簡単な方法):
- `aws configure` を実行します。
- アクセスキー、シークレットキー、デフォルトのリージョン (そのリージョンで Bedrock にアクセスできることを確認してください) を入力し、出力を json に設定します。
- **手動設定**:
- `~/.aws/credentials` および `~/.aws/config` に認証情報ファイルを作成します

IAM ロール/ユーザーには、Amazon Bedrock、特に使用しようとしている Claude 3.7 Sonnet および Claude 3.5 Haiku モデルにアクセスする権限が必要です。

## 重要: プロンプト キャッシュとコスト管理

⚠️ **トークン使用に関する警告**: Claude Code はトークンを大量に消費するため、特にプロンプ​​ト キャッシュが有効になっていない場合、使用コストが高くなる可能性があります。

### プロンプト キャッシュについて:
- **機能**: プロンプトをキャッシュすることで、トークンの使用とコストを大幅に削減します
- **可用性**: プレビューで一部の Bedrock 顧客が利用できます (将来 Bedrock GA に導入予定)
- **制御方法**: `DISABLE_PROMPT_CACHING` 環境変数を使用します:
- キャッシュを無効にするには `1` に設定します
- キャッシュを有効にするには、この変数を省略するか `0` に設定します (アクセス権がある場合)

### 推奨事項:
- プロンプト キャッシュにアクセスできる場合 (プレビューまたは GA): 必ず使用してください。`DISABLE_PROMPT_CACHING` 変数を省略するか `0` に設定します。
- プロンプト キャッシュがない場合: コストが急速に蓄積される可能性があるため、使用量に注意してください。
- Claude Code 内の `/cost` コマンドを使用して使用量を監視します
- コンテキスト サイズを管理するために、`/compact` コマンドと `/clear` コマンドを定期的に使用することを検討してください

## Amazon Bedrock で Claude Code を設定する

わずか数個のコマンドで、Claude Code を Amazon Bedrock に簡単に接続できます。

In [ ]:
# Install Claude Code
!npm install -g @anthropic-ai/claude-code

## ターミナル ウィンドウで Claude Code を使用する

Claude Code を最大限に活用するには、ターミナル ウィンドウを開いてツールを直接操作してください。これにより、次のことが可能になります:

1. Claude Code コマンドを対話的に実行する
2. リアルタイムの応答と提案を表示する
3. コードベースを効率的にナビゲートする
4. Claude Code の AI 支援開発の全機能を体験する

## 環境変数を構成する

**サンドボックス ディレクトリに変更する**

Claude Code プロジェクト用のクリーンで整理された環境を確保するには、このノートブックの横にあるサンドボックス ディレクトリに移動します。これにより、プロジェクトを分離して簡単にアクセスできるようになります。

ターミナルで次のコマンドを実行して、サンドボックス ディレクトリに移動します:

```bash
cd sandbox
```

これらのコマンドをコピーしてターミナルに貼り付け、Amazon Bedrock で Claude Code を構成します:

```bash
# Claude Code が Amazon Bedrock を使用できるようにします

export CLAUDE_CODE_USE_BEDROCK=1

# Bedrock で Claude 3.7 Sonnet モデルを指定します

export ANTHROPIC_MODEL='us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# この機能にアクセスできない場合は、プロンプト キャッシュを無効にします

export DISABLE_PROMPT_CACHING=1
```

これらの環境変数を設定したら、次のコマンドを実行して Claude Code を起動できます:

```bash
claude
```

![Claude Code Bedrock 構成](images/0.jpg)

## `/init` によるプロジェクトの初期化

Claude Code は、新しいプロジェクトを始めるのに役立つコマンド `/init` を提供しています。

### `/init` の機能:
- プロジェクト用の包括的な `CLAUDE.md` ガイドを生成します
- 特定のコードベースに合わせたドキュメントを作成します
- 新しいチーム メンバーがプロジェクト構造を理解できるようにします
- プロジェクトで Claude Code を使用するためのリファレンスを提供します

### `/init` の使用方法:
1. プロジェクト ディレクトリで `claude` を使用して Claude Code を起動します
2. 対話型 REPL で `/init` と入力します
3. Claude がコードベースを分析し、ガイドを生成します
4. 生成された `CLAUDE.md` ファイルを確認します
5. 必要に応じて `claude commit` を使用してリポジトリにファイルをコミットします

生成されたガイドは、プロジェクトのアーキテクチャ、主要なコンポーネント、ワークフローを説明するライブ ドキュメントとして機能します。これにより、誰でもプロジェクトを理解して貢献しやすくなります。コードベース。

![Claude Code Init](images/1.jpg)

## `/init` コマンドの動作を理解する

Claude Code の `/init` コマンドは、コードベースを包括的に分析して、役立つドキュメントを作成します。このコマンドを実行すると、リポジトリの標準化された「マップ」も作成されます。これにより、Claude は、リポジトリまたはコンテキスト ウィンドウを継続的に検索して重要な高レベルの情報を探す必要がなくなり、トークンを節約できます。実行されると、次の処理が行われます。

1. リポジトリ構造をスキャンし、ファイルとその関係を識別します

2. 既存のドキュメントを調べます (デモをブートストラップするためにサンドボックス フォルダーに配置した calculator.md ファイルなど)

3. 構成ファイル、コーディング パターン、プロジェクト構成を探します

4. 既存のガイドライン (カーソル ルールや Copilot の指示など) をチェックします

この分析に基づいて、Claude Code は次の内容を含む CLAUDE.md ファイルを生成します:
- プロジェクトに合わせた実用的なビルドおよび実行コマンド
- テクノロジー スタックに固有のコード スタイル ガイドライン
- 既存のパターンに一致するアーキテクチャの推奨事項
- 一貫性を維持するためのドキュメント標準

この自動化されたドキュメントにより、人間の開発者と AI アシスタントの両方がプロジェクトの構造と規則を理解し、コラボレーションをより効率的に行うことができます。

## Claude Code からのコード追加の承認とレビュー

Claude Code がコードの変更や追加を提案した場合、承認する前に慎重に確認することが重要です。

### 権限システム
- Claude Code は、安全のために人間が関与する権限システムを使用しています
- ファイルの変更には、デフォルトで明示的な承認が必要です
- ファイル変更の「次回から確認しない」プロンプトに同意しない限り、変更を行う前に確認を求められます

### コード レビューのベスト プラクティス
1. 承認する前に、**提案された変更を慎重に検討する**
2. **コードの動作を理解する** - 必要に応じて Claude に説明を求めます
3. 考慮されていない可能性のある**エッジ ケースを確認する**
4. 機密データを処理するコードの**セキュリティへの影響を確認する**
5. 適用後に**変更をテストする** して、期待どおりに動作することを確認します

### 承認オプション
Claude がファイルの変更を提案した場合、次のようなオプションが表示されます。
- **はい**: この特定のアクションを 1 回承認します
- **はい、今後は確認しません**: セッションの残りの間、このタイプのアクションを承認します
- **いいえ**: 提案された変更を拒否します

> ⚠️ **重要** ⚠️: Claude によって生成されたすべてのコード、ファイルの変更、およびコマンドが人間によって厳密にレビューされるようにするには、常に **はい** オプションを使用し、「はい、今後は確認しません」を使用しないでください。この人間が関与するレビュー プロセスは、セキュリティとコードの品質を維持するために不可欠です。 👁️ 🔒

Claude Code セッションを終了すると、ファイル変更権限がリセットされ、追加の安全レイヤーが提供されることに注意してください。

![Claude Code 権限要求の例](images/2.jpg)

## 計算機 Web アプリケーションの構築

Claude Code の機能を調べ、プロジェクト ドキュメントを確認したので、Claude に計算機 Web アプリケーションの構築を依頼しましょう。

### Claude にサイトの作成を依頼しましょう:

```
> では、サイトを作成しましょう。calculator.md の要件に基づいて、CLAUDE.md のガイドラインに従って計算機 Web アプリケーションを実装してください。機能的な計算機を作成するには、HTML、CSS、および JavaScript コンポーネントが必要です。HTML ファイルに calculator.html という名前を付けます。
```

Claude が要件を分析し、次の作成をガイドします:

1. 計算機構造の HTML コンポーネント

2. スタイルとレスポンシブ デザイン用の CSS コンポーネント

3. 計算機機能用の JavaScript コンポーネント

Claude が提案する各ファイルを確認し、変更を承認できます。前に説明したコード レビューのベスト プラクティスを使用することを忘れないでください。

![Claude Code Calculator](images/3.jpg)

## 計算機 Web アプリケーションのテスト

Claude が計算機 Web アプリケーションを作成したので、テストしてみましょう。

### 計算機を開く

1. 計算機ファイルが作成された `sandbox` ディレクトリに移動します

2. Web ブラウザーで `calculator.html` ファイルを開きます
- ファイルをダブルクリックするか、右クリックして [使用するブラウザーで開く] を選択することでこれを行うことができます

### 機能のテスト

さまざまな計算を実行して、すべてが正しく機能することを確認します:
- 基本的な算術演算 (加算、減算、乗算、除算)
- クリア機能 (C ボタン)
- 削除機能 (DEL ボタン)
- ブラウザー ウィンドウのサイズを変更して、レスポンシブ デザインをテストします

### 潜在的な機能強化

要件からのカスタマイズのアイデアを覚えておいてください:
- 科学計算機機能
- 計算履歴
- テーマの切り替え
- 複雑な式に対する括弧のサポート

![Claude Code Calculator](images/4.jpg)

## /compact コマンドによるコンテキストの管理

`/compact` コマンドは、コンテキストを管理し、Claude Code との効率的な会話を維持するための強力なツールです。

### /compact の仕組み:
- 会話履歴を簡潔な形式で要約します
- トークン数を減らしながら重要な情報を保持します
- 完全な会話履歴がなくてもコンテキスト認識を維持します
- コマンド `/compact` を使用して Claude Code ターミナルで直接実行します

### /compact を使用する利点:
- **一貫性を維持**: `/clear` とは異なり、会話履歴の重要な部分を保持します
- **時間的一貫性を強化**: Claude が対話のタイムラインとシーケンスをよりよく理解できるようにします
- **コンテキスト参照を改善**: Claude が会話の以前の情報を正確に参照して思い出すことが容易になります
- **トークン使用量を削減**: 消費されるトークンの数が大幅に削減され、コスト管理に役立ちます
- **パフォーマンスを向上**: 処理するコンテキストが少なくなるため、Claude はより迅速に応答できます
- **会話の長さを延長**:コンテキスト制限に達することなく、より長く生産性の高いセッションを実現できます

### プロンプト キャッシュと /compact:
- **プロンプト キャッシュあり**: Bedrock でプロンプト キャッシュが有効になっている場合、プロンプトを繰り返しても課金されないため、`/compact` のコスト メリットはそれほど大きくありません
- **主なメリットの変化**: プロンプト キャッシュを使用すると、`/compact` はコスト管理よりも会話の一貫性の維持に価値が高まります
- **Anthropic と Bedrock**: Anthropic の Claude サービスと Bedrock の Claude 実装はどちらもプロンプト キャッシュを提供していますが、それぞれ異なるアクティベーション方法で独立して動作します
- **引き続き推奨**: プロンプト キャッシュが有効になっている場合でも、`/compact` を定期的に使用することで、Claude は会話の流れとプロジェクトのコンテキストをより明確に把握できます

### /compact を使用する場合:
- 主要なタスクまたはマイルストーンを完了した後
- Claude の応答が遅くなっていることに気付いたとき
- 新しい、関連するサブタスク
- Claude が過去のイベントや指示の順序を混乱しているように見える場合
- プロジェクトのさまざまな側面間を移行する場合
- 長時間のコーディング セッション中に定期的に (プロンプトのキャッシュ状態に関係なく)

`/compact` を定期的に使用することは、Claude Code との一貫性のある効率的なやり取りを維持するためのベスト プラクティスです。特に、長い会話を必要とする複雑なコーディング タスクの場合に効果的です。

これを実践してみましょう。Claude Code ターミナルで `/compact` コマンドを使用して、計算機プロジェクトに関する会話を要約してみてください。これにより、コンテキストを維持しながらトークンの使用を減らし、次のタスクを効率的に進めることができます。`/compact` を使用した後、Claude がプロジェクトを認識しながら、より合理化された会話エクスペリエンスを提供していることに注目してください。

![Claude Code Compact](images/5.jpg)

## Claude Code によるプロジェクトの強化: 科学計算機能の追加

Claude Code は、既存のプロジェクトを総合的に理解して変更することに優れています。`/compact` コマンドを使用して会話のコンテキストを最適化した後、Claude Code が新しい機能で計算アプリケーションを強化するのにどのように役立つかを見てみましょう。

### 既存のコードベースに変更を加える

Claude Code を使用する場合、自然言語で変更したい点を簡単に説明できます。たとえば、プロンプト「オプションの科学計算機能を追加する」を使用します。

![Claude Code 科学計算機能](images/6.jpg)

## 拡張計算機のテスト

これで、アプリケーションに科学計算機の機能が追加されました。次はテストです!

### 計算機のテスト方法:

1. Web ブラウザーで `calculator.html` ファイルを開きます

2. 基本的な算術演算 (加算、減算、乗算、除算) を試します

3. 新しく追加された科学関数 (sin、cos、tan、log など) をテストします

4. すべての計算で期待どおりの結果が生成されることを確認します

5. UI がデバイスに正しく表示されることを確認します

ファイルはファイル システムから直接開くことができます。ファイルを保存した場所に移動してダブルクリックするか、右クリックして [使用するブラウザーで開く] を選択します。

時間をかけてさまざまな計算を試し、すべてが期待どおりに動作することを確認してください!

![Claude Code 科学計算機](images/7.jpg)

## Claude Code によるコストの監視

Claude Code では、トークンの使用状況と関連コストを追跡する方法がいくつか用意されています。一般的な使用量は開発者 1 人あたり 1 日 5 ～ 10 ドルですが、集中的に使用すると 1 時間あたり 100 ドルを超えることもあるため、これらのコストを理解して管理することは不可欠です。

### Amazon Bedrock での Claude Code の料金

Amazon Bedrock で Claude Code を使用する場合、料金は Bedrock の標準トークンベースの料金モデルに従います。Claude Code は Claude 3.7 Sonnet と Claude 3.5 Haiku の両方のモデルを使用するため、AWS アカウントで両方にアクセスできる必要があります。固定された入力と出力 (特定のケース) を伴う従来の LLM インタラクションとは異なり、コード エージェントはオープンエンド方式で動作し、次の場合に大量のトークンを消費する可能性があります。

- コードベースを探索して分析し、コンテキストを理解する
- 最適なソリューション アプローチを提示する前に、複数のソリューション アプローチを内部で生成する
- コマンドを実行し、その出力を処理する
- 会話履歴を維持して、セッション全体で一貫した支援を確保する
- 複雑な問題解決のために拡張思考を実行する

### コストの追跡:

- **セッション中**: `/cost` コマンドを使用して、現在のトークン使用量と推定コストを確認する
- **セッションの概要**: Claude Code を終了するときに自動的に表示されるコストの概要を表示する
- **履歴データ**: Anthropic Console で使用履歴を確認する
- **支出制限を設定する**: アカウント設定で支出しきい値を構成する

### トークン使用量を削減するための戦略:

- **会話を圧縮する**: コンテキストが大きくなると `/compact` を使用する
- **特定のクエリを作成する**: 回避する不必要なファイル スキャンをトリガーするあいまいなリクエスト
- **複雑なタスクを分割する**: 大きなタスクを焦点を絞ったインタラクションに分割する
- **タスク間の履歴をクリアする**: トピックを切り替えるときに `/clear` を使用してコンテキストをリセットする

コストは、コードベースのサイズ、クエリの複雑さ、分析対象のファイル数、会話の長さによって大幅に異なる場合があることに注意してください。チーム展開の場合、小規模なパイロット グループから始めると、より広範な展開の前に使用パターンを確立するのに役立ちます。

![Claude Code Cost](images/8.jpg)

## 結論: Amazon Bedrock で Claude Code のパワーを解き放つ

Amazon Bedrock 上の Claude Code は、AI 支援ソフトウェア開発における大きな進歩を表しています。 Anthropic の最先端の Claude 3.7 Sonnet モデルと、AWS Bedrock のセキュリティ、スケーラビリティ、エンタープライズ グレードのインフラストラクチャを組み合わせることで、開発者は次のような強力なコーディング アシスタントを利用できるようになります。

- 複雑なコードベースを理解してナビゲートする
- 複数のファイルにわたるコードを生成、編集、デバッグする
- アーキテクチャと実装に関する詳細な質問に答える
- 開発環境でコマンドを実行し、実際のアクションを実行する
- 長時間のコーディング セッション全体を通じてコン​​テキスト認識を維持する

### Bedrock での Claude Code の主な利点:

- **エンタープライズ セキュリティ**: コードのやり取りを AWS インフラストラクチャ内に維持する
- **コスト管理**: 組み込みツールを使用してトークンの使用状況を追跡および最適化する
- **シームレスな統合**: 既存の AWS Bedrock の割り当てと課金を活用する
- **開発者エクスペリエンス**: 最小限のセットアップでターミナルから直接作業する
- **コンテキスト管理**: `/compact` などのコマンドを使用して生産性の高いセッションを維持する

### 次のステップ:

1. **小規模から始める**: Claude Code に慣れるために、集中的なタスクから始めましょう
2. **ワークフローを確立する**: AI との効果的なコラボレーションのためのパターンを開発する
3. **使用状況を監視**: コストを監視し、`/compact` やその他の戦略で最適化する
4. **知識を共有する**: チームと効果的なプロンプトとアプローチを文書化する
5. **最新情報を入手する**: Claude Code は定期的に改善しながら積極的に開発されています

このガイドで概説されているセットアップ手順とベストプラクティスに従うことで、Amazon Bedrock 上の Claude Code の可能性を最大限に活用して開発プロジェクトに取り組むことができます。

Claude Code は、開発者としての能力を強化するように設計されており、専門知識を置き換えるものではないことに注意してください。最も効果的な使用方法は、明確な指示を与え、AI の提案を慎重に確認するコラボレーションアプローチです。